In [1]:
import Stemmer

In [2]:
stemmer = Stemmer.Stemmer('english')

In [3]:
import numpy as np

In [4]:
train=np.load('./datasets/processed/train-twitter.npy',allow_pickle=True)

In [5]:
test=np.load('./datasets/processed/test-twitter.npy',allow_pickle=True)

In [6]:
train.shape

(6248,)

In [10]:
test.shape

(692,)

In [11]:
alldata=np.concatenate((train,test))

In [12]:
alldata.shape

(6940,)

In [13]:
target_list=[]
stem_list=[]
for data in alldata:
    target_list.append(data['text_aspect'].strip())
    s=stemmer.stemWord(data['text_aspect'].strip())
#     target_list.append(s)
#     stem_list
    stem_list.append(s)


In [14]:
len(set(target_list))

118

In [15]:
target_list=stem_list

In [16]:
len(set(stem_list))

118

In [17]:
len(target_list)

6940

In [18]:
target_set=set(target_list)

In [19]:
len(target_set)

118

# split

In [19]:
target_list=[]
stem_list=[]
for data in alldata:
    target_list.append(data['text_aspect'])
    s=stemmer.stemWord(data['text_aspect'])
#     target_list.append(s)
#     stem_list
    stem_list.append(s)
target_list=stem_list

In [20]:
target_set=set(target_list)

In [22]:
len(target_set)

118

In [23]:
target_dict=dict()
for i in range(len(alldata)):
    target=alldata[i]['text_aspect']
    if target not in target_dict:
        l=list()
        l.append(i)
        target_dict[target]=l
    else:
        l=target_dict[target]
        l.append(i)
        target_dict[target]=l
        

        
    

In [28]:
train_val_dict = {key: value for i, (key, value) in enumerate(target_dict.items()) if i % 10 != 0}
test_dict = {key: value for i, (key, value) in enumerate(target_dict.items()) if i % 10 == 0}

In [29]:
test_index=[]
for key,value in test_dict.items():
    test_index.extend(value)
len(test_index)

1086

In [30]:
np.save('./datasets/resplit/test-twitter-pair.npy',alldata[test_index])

In [36]:
train_val_index=[]
for key,value in train_val_dict.items():
    train_val_index.extend(value)
len(train_val_index)

5854

# compose new sentence pair for training test

In [31]:
import logging
import argparse
import math
import os
import sys
from time import strftime, localtime
import random
import numpy
import numpy as np
from manager_torch import *

In [32]:
import torch
from data_utils import build_tokenizer, build_embedding_matrix, Tokenizer4Bert, ABSADataset

/home/xiangpan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xiangpan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xiangpan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xiangpan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.

In [33]:
parser = argparse.ArgumentParser()
parser.add_argument('--model_name', default='bert_spc', type=str)
parser.add_argument('--dataset', default='laptop', type=str, help='twitter, restaurant, laptop')
parser.add_argument('--optimizer', default='adam', type=str)
parser.add_argument('--initializer', default='xavier_uniform_', type=str)
parser.add_argument('--learning_rate', default=2e-5, type=float, help='try 5e-5, 2e-5 for BERT, 1e-3 for others')
parser.add_argument('--dropout', default=0.1, type=float)
parser.add_argument('--l2reg', default=0.01, type=float)
parser.add_argument('--num_epoch', default=10, type=int, help='try larger number for non-BERT models')
parser.add_argument('--batch_size', default=16, type=int, help='try 16, 32, 64 for BERT models')
parser.add_argument('--log_step', default=5, type=int)
parser.add_argument('--embed_dim', default=300, type=int)
parser.add_argument('--hidden_dim', default=300, type=int)
parser.add_argument('--bert_dim', default=768, type=int)
parser.add_argument('--pretrained_bert_name', default='bert-base-uncased', type=str)
parser.add_argument('--max_seq_len', default=128, type=int)
parser.add_argument('--polarities_dim', default=3, type=int)
# parser.add_argument('--hops', default=3, type=int)
parser.add_argument('--device', default='cuda:1', type=str, help='e.g. cuda:0')
parser.add_argument('--seed', default=None, type=int, help='set seed for reproducibility')
parser.add_argument('--valset_ratio', default=0, type=float, help='set ratio between 0 and 1 for validation support')
parser.add_argument('--load_mode', default=0, type=int, help='load existed model')

parser.add_argument('--can', default=0, type=float, help='using tfm')
parser.add_argument('--adv', default=0, type=float, help='using adv training')
parser.add_argument('--aux', default=0, type=float, help='using aux training')

parser.add_argument('--domain', default=0, type=str, help='using domain bert')

# The following parameters are only valid for the lcf-bert model
parser.add_argument('--local_context_focus', default='cdm', type=str, help='local context focus mode, cdw or cdm')
parser.add_argument('--SRD', default=3, type=int, help='semantic-relative-distance, see the paper of LCF-BERT model')



opt = parser.parse_args([])

In [34]:
def pad(a,maxlen):
    B = np.pad(a, (0, maxlen - len(a)%maxlen), 'constant')
    return B

In [35]:
tokenizer = Tokenizer4Bert(opt.max_seq_len, opt.pretrained_bert_name)
for i in range(20):
    b='['+str(i)+'b]'
    e='['+str(i)+'e]'
    num_added_tokens = tokenizer.add_tokens([b,e])      
    
def pad_and_truncate(sequence, maxlen, dtype='int64', padding='post', truncating='post', value=0):
    x = (np.ones(maxlen) * value).astype(dtype)
    if truncating == 'pre':
        trunc = sequence[-maxlen:]
    else:
        trunc = sequence[:maxlen]
    trunc = np.asarray(trunc, dtype=dtype)
    if padding == 'post':
        x[:len(trunc)] = trunc
    else:
        x[-len(trunc):] = trunc
    return x

In [38]:
train_val_target_dict=dict()
for i in range(len(alldata)):
    if i in train_val_index:
        target=alldata[i]['text_aspect']
        if target not in train_val_target_dict:
            l=list()
            l.append(i)
            train_val_target_dict[target]=l
        else:
            l=train_val_target_dict[target]
            l.append(i)
            train_val_target_dict[target]=l
        

        
    

In [39]:
new_all=[]
count=0
for key,val in train_val_target_dict.items():
    print(key,val,len(val))
    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!',count)
    count+=1
    last_j=0
    for i in val:
        if i>last_j:
#             print(i)
    #         multi_target_indices=alldata[i]['multi_target_indices']
    #         multi_target_segments_ids=alldata[i]['multi_target_segments_ids']
            target_pos_1=alldata[i]['target_pos']
    #         poss=alldata[i]['poss']
    #         polarity_list=alldata[i]['polarity_list']
            polarity_1=alldata[i]['polarity']
#             print(alldata[i])
            text_multi_1=alldata[i]['text_multi']
            text_aspect_1=alldata[i]['text_aspect']
            paired=False
            for j in val:
                if j>i:   # ignore self
                    print(i,j)
                    text_multi_2=alldata[j]['text_multi']
                    target_pos_2=alldata[j]['target_pos']
                    polarity_2=alldata[j]['polarity']

                    text_multi_2=text_multi_2.replace('[CLS]','')
                    text_multi_2=text_multi_2.replace('[0b]','[1b]')
                    text_multi_2=text_multi_2.replace('[0e]','[1e]')

    #                 print(text_multi_2)
                    text_multi=text_multi_1+text_multi_2
                    multi_target_indices = tokenizer.text_to_sequence(text_multi)
                    multi_target_indices_1 = tokenizer.text_to_sequence(text_multi_1)
                    multi_target_indices_2 = tokenizer.text_to_sequence(text_multi_2)
                    multi_target_segments_ids=np.asarray([0] * np.sum(multi_target_indices_1 != 0)+[1] * (np.sum(multi_target_indices_2 != 0)))
                    multi_target_segments_ids = pad_and_truncate(multi_target_segments_ids, tokenizer.max_seq_len)

                    target_pos_2=target_pos_2+np.sum(multi_target_indices_1 != 0)-1


                    poss= [target_pos_1,target_pos_2]
                    poss=pad(poss,maxlen=20)
                    poss=torch.tensor(poss)

                    polarity_list=[polarity_1+1,polarity_2+1]
                    polarity_list=pad(polarity_list,maxlen=20)
                    polarity_list=torch.tensor(polarity_list)
    #                 print(multi_target_indices)
                    new={
                        'text_aspect':text_aspect_1,
                        'text_multi':text_multi,
                        'multi_target_indices':multi_target_indices,
                        'multi_target_segments_ids':multi_target_segments_ids,
                        'target_pos':target_pos_1,
                        'poss':poss,
                        'polarity_list':polarity_list,
                        'polarity':polarity_1,
                    }
                    new_all.append(new)
#                     print(len(new_all))
                    break
                last_j=j

britney spears [1, 5, 9, 17, 34, 39, 48, 49, 57, 58, 64, 65, 77, 83, 98, 102, 110, 125, 141, 151, 179, 190, 193, 195, 203, 212, 240, 256, 261, 265, 274, 275, 285, 286, 294, 303, 308, 315, 325, 328, 335, 343, 347, 351, 355, 360, 362, 367, 373, 376, 378, 381, 392, 393, 397, 402, 405, 413, 425, 434, 435, 447, 450, 454, 475, 484, 489, 501, 509, 520, 524, 528, 529, 533, 541, 543, 549, 554, 561, 563, 567, 568, 572, 581, 594, 595, 598, 599, 600, 601, 616, 618, 635, 643, 652, 659, 668, 680, 681, 682, 695, 700, 712, 717, 722, 741, 743, 749, 755, 758, 767, 774, 778, 814, 825, 830, 846, 866, 870, 872, 877, 879, 883, 893, 896, 899, 902, 910, 911, 913, 914, 915, 938, 950, 954, 978, 981, 985, 989, 995, 1002, 1017, 1021, 1027, 1029, 1037, 1046, 1048, 1051, 1055, 1059, 1060, 1069, 1083, 1086, 1089, 1090, 1095, 1098, 1103, 1123, 1130, 1134, 1140, 1145, 1152, 1171, 1173, 1178, 1199, 1207, 1226, 1227, 1256, 1258, 1272, 1273, 1287, 1290, 1299, 1300, 1302, 1309, 1318, 1322, 1334, 1341, 1346, 1349, 1366, 13

1863 1869
1869 1871
1871 1873
1873 1881
1881 1892
1892 1896
1896 1898
1898 1900
1900 1923
1923 1927
1927 1933
1933 1934
1934 1937
1937 1946
1946 1948
1948 1949
1949 1954
1954 1958
1958 1962
1962 1964
1964 1969
1969 1973
1973 1975
1975 1988
1988 2001
2001 2009
2009 2025
2025 2046
2046 2053
2053 2071
2071 2081
2081 2086
2086 2094
2094 2099
2099 2106
2106 2111
2111 2115
2115 2128
2128 2136
2136 2145
2145 2147
2147 2150
2150 2162
2162 2171
2171 2173
2173 2179
2179 2194
2194 2200
2200 2214
2214 2236
2236 2265
2265 2272
2272 2275
2275 2278
2278 2297
2297 2301
2301 2305
2305 2307
2307 2313
2313 2325
2325 2327
2327 2330
2330 2332
2332 2336
2336 2338
2338 2340
2340 2345
2345 2351
2351 2359
2359 2364
2364 2369
2369 2372
2372 2388
2388 2393
2393 2407
2407 2410
2410 2424
2424 2426
2426 2433
2433 2445
2445 2455
2455 2462
2462 2465
2465 2467
2467 2468
2468 2474
2474 2477
2477 2478
2478 2483
2483 2487
2487 2497
2497 2498
2498 2514
2514 2516
2516 2518
2518 2521
2521 2528
2528 2529
2529 2544
2544 2548


2700 2721
2721 3087
3087 3149
3149 3211
3211 3247
3247 3307
3307 3442
3442 3549
3549 3644
3644 3677
3677 3712
3712 3751
3751 3923
3923 3969
3969 3983
3983 4033
4033 4047
4047 4119
4119 4200
4200 4311
4311 4436
4436 4513
4513 4554
4554 4603
4603 4619
4619 4799
4799 4851
4851 4882
4882 5286
5286 5298
5298 5430
5430 5552
5552 5674
5674 5675
5675 5739
5739 5881
5881 5921
5921 5947
5947 6034
6034 6084
6084 6464
6464 6597
6597 6744
6744 6844
6844 6875
6875 6916
wii [3, 7, 21, 38, 74, 89, 153, 162, 185, 230, 237, 255, 270, 283, 298, 305, 322, 333, 387, 461, 466, 474, 512, 517, 557, 586, 611, 655, 657, 689, 690, 783, 804, 869, 939, 957, 963, 964, 977, 1019, 1035, 1047, 1111, 1135, 1147, 1158, 1253, 1314, 1418, 1453, 1507, 1520, 1542, 1594, 1638, 1646, 1874, 1877, 1901, 1921, 1928, 1935, 2065, 2105, 2125, 2166, 2254, 2299, 2306, 2346, 2430, 2545, 2620, 2634, 2635, 2665, 2691, 2692, 2723, 2796, 2813, 2830, 2847, 2873, 2915, 2916, 2927, 2988, 2991, 3020, 3042, 3065, 3142, 3154, 3167, 3173, 3177, 

1127 1150
1150 1151
1151 1170
1170 1184
1184 1213
1213 1215
1215 1219
1219 1222
1222 1225
1225 1237
1237 1242
1242 1263
1263 1267
1267 1270
1270 1313
1313 1323
1323 1338
1338 1356
1356 1363
1363 1369
1369 1375
1375 1471
1471 1482
1482 1489
1489 1516
1516 1518
1518 1524
1524 1526
1526 1540
1540 1553
1553 1571
1571 1579
1579 1667
1667 1679
1679 1711
1711 1730
1730 1731
1731 1734
1734 1767
1767 1770
1770 1780
1780 1821
1821 1826
1826 1827
1827 1828
1828 1833
1833 1845
1845 1879
1879 1886
1886 1925
1925 1956
1956 2012
2012 2029
2029 2033
2033 2047
2047 2083
2083 2089
2089 2095
2095 2133
2133 2134
2134 2157
2157 2178
2178 2206
2206 2207
2207 2213
2213 2242
2242 2247
2247 2250
2250 2264
2264 2331
2331 2355
2355 2363
2363 2447
2447 2451
2451 2470
2470 2505
2505 2512
2512 2520
2520 2530
2530 2547
2547 2572
2572 2639
2639 2655
2655 2664
2664 2694
2694 2698
2698 2753
2753 2762
2762 2779
2779 2785
2785 2787
2787 2823
2823 2837
2837 2903
2903 2919
2919 2933
2933 2937
2937 2958
2958 2965
2965 3039


936 944
944 960
960 968
968 972
972 1004
1004 1032
1032 1108
1108 1148
1148 1176
1176 1201
1201 1208
1208 1257
1257 1261
1261 1340
1340 1444
1444 1468
1468 1556
1556 1564
1564 1668
1668 1702
1702 1708
1708 1724
1724 1862
1862 1864
1864 1936
1936 1943
1943 1966
1966 1980
1980 2103
2103 2116
2116 2322
2322 2352
2352 2380
2380 2382
2382 2444
2444 2645
2645 2652
2652 2752
2752 2832
2832 2904
2904 2943
2943 3004
3004 3016
3016 3040
3040 3044
3044 3096
3096 3116
3116 3164
3164 3192
3192 3284
3284 3308
3308 3344
3344 3352
3352 3432
3432 3454
3454 3483
3483 3540
3540 3564
3564 3574
3574 3632
3632 3724
3724 3840
3840 3842
3842 3932
3932 3972
3972 4034
4034 4097
4097 4216
4216 4300
4300 4307
4307 4384
4384 4428
4428 4510
4510 4706
4706 4740
4740 4760
4760 4820
4820 4825
4825 4924
4924 4944
4944 4984
4984 5020
5020 5108
5108 5139
5139 5148
5148 5168
5168 5176
5176 5208
5208 5256
5256 5334
5334 5382
5382 5392
5392 5403
5403 5518
5518 5600
5600 5636
5636 5640
5640 5820
5820 5840
5840 6008
6008 6032

...

1023 1058
1058 1097
1097 1238
1238 1245
1245 1286
1286 1393
1393 1441
1441 1448
1448 1517
1517 1558
1558 1589
1589 1595
1595 1615
1615 1691
1691 1913
1913 1942
1942 1976
1976 1986
1986 2010
2010 2015
2015 2034
2034 2042
2042 2169
2169 2219
2219 2285
2285 2365
2365 2454
2454 2459
2459 2491
2491 2571
2571 2650
2650 2703
2703 2797
2797 2803
2803 2822
2822 2857
2857 2917
2917 2923
2923 2969
2969 3086
3086 3120
3120 3165
3165 3278
3278 3285
3285 3287
3287 3333
3333 3401
3401 3465
3465 3678
3678 3691
3691 3801
3801 3839
3839 3886
3886 3939
3939 4014
4014 4055
4055 4146
4146 4175
4175 4202
4202 4205
4205 4242
4242 4273
4273 4281
4281 4293
4293 4331
4331 4371
4371 4666
4666 4849
4849 4893
4893 4913
4913 4914
4914 4919
4919 4941
4941 4965
4965 4989
4989 5002
5002 5005
5005 5060
5060 5106
5106 5163
5163 5178
5178 5206
5206 5215
5215 5223
5223 5225
5225 5258
5258 5381
5381 5387
5387 5389
5389 5447
5447 5511
5511 5534
5534 5543
5543 5620
5620 5630
5630 5646
5646 5706
5706 5709
5709 5710
5710 5735


980 1005
1005 1033
1033 1036
1036 1092
1092 1093
1093 1117
1117 1125
1125 1129
1129 1180
1180 1188
1188 1224
1224 1276
1276 1289
1289 1357
1357 1485
1485 1528
1528 1565
1565 1625
1625 1644
1644 1652
1652 1701
1701 1717
1717 1748
1748 1820
1820 1941
1941 1993
1993 2057
2057 2092
2092 2140
2140 2177
2177 2196
2196 2205
2205 2209
2209 2221
2221 2229
2229 2253
2253 2288
2288 2293
2293 2385
2385 2409
2409 2417
2417 2504
2504 2585
2585 2605
2605 2637
2637 2644
2644 2828
2828 2936
2936 2972
2972 2978
2978 3064
3064 3181
3181 3209
3209 3252
3252 3253
3253 3301
3301 3321
3321 3356
3356 3389
3389 3409
3409 3421
3421 3497
3497 3529
3529 3581
3581 3720
3720 3748
3748 3822
3822 3849
3849 3868
3868 3893
3893 3908
3908 3924
3924 4000
4000 4021
4021 4072
4072 4077
4077 4129
4129 4148
4148 4208
4208 4241
4241 4269
4269 4277
4277 4360
4360 4377
4377 4378
4378 4529
4529 4533
4533 4613
4613 4724
4724 4857
4857 4916
4916 4945
4945 4952
4952 5017
5017 5053
5053 5057
5057 5069
5069 5081
5081 5177
5177 5229
5

4804 4969
4969 5028
5028 5315
5315 5348
5348 5492
5492 5500
5500 6497
6497 6608
6608 6626
6626 6795
steve jobs [91, 94, 112, 127, 218, 280, 479, 685, 747, 772, 906, 999, 1000, 1102, 1167, 1232, 1502, 1538, 1547, 1710, 1791, 1834, 1855, 1882, 1885, 1903, 1982, 1990, 2006, 2018, 2067, 2126, 2158, 2390, 2414, 2463, 2598, 2791, 2814, 2815, 2911, 2999, 3088, 3094, 3122, 3203, 3224, 3226, 3230, 3303, 3354, 3403, 3439, 3534, 3551, 3647, 3651, 3656, 3827, 3942, 3958, 3986, 4053, 4140, 4142, 4275, 4276, 4352, 4388, 4478, 4495, 4540, 4772, 4824, 4843, 4850, 4871, 4931, 4966, 4967, 5064, 5091, 5125, 5171, 5299, 5322, 5351, 5435, 5490, 5594, 5787, 5789, 5831, 5858, 6007, 6081, 6138, 6170, 6323, 6354, 6383, 6395, 6466, 6506, 6606, 6735, 6858, 6867, 6878, 6938] 110
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 34
91 94
94 112
112 127
127 218
218 280
280 479
479 685
685 747
747 772
772 906
906 999
999 1000
1000 1102
1102 1167
1167 1232
1232 1502
1502 1538
1538 1547
1547 1710
1710 1791
1791 1834
1834 1855
1855 1882


173 178
178 199
199 221
221 239
239 262
262 264
264 344
344 419
419 439
439 477
477 480
480 573
573 596
596 624
624 628
628 629
629 661
661 677
677 688
688 701
701 781
781 790
790 848
848 881
881 962
962 1105
1105 1137
1137 1153
1153 1165
1165 1166
1166 1185
1185 1193
1193 1202
1202 1212
1212 1234
1234 1244
1244 1265
1265 1321
1321 1345
1345 1365
1365 1383
1383 1413
1413 1501
1501 1567
1567 1609
1609 1629
1629 1648
1648 1651
1651 1664
1664 1682
1682 1685
1685 1725
1725 1729
1729 1786
1786 1797
1797 1825
1825 1887
1887 1891
1891 1995
1995 2003
2003 2005
2005 2024
2024 2045
2045 2093
2093 2154
2154 2190
2190 2198
2198 2233
2233 2274
2274 2371
2371 2449
2449 2461
2461 2554
2554 2579
2579 2651
2651 2658
2658 2707
2707 2733
2733 2737
2737 2767
2767 2843
2843 2871
2871 2888
2888 2890
2890 2909
2909 2955
2955 2971
2971 2980
2980 2986
2986 3027
3027 3030
3030 3195
3195 3197
3197 3237
3237 3251
3251 3269
3269 3309
3309 3346
3346 3360
3360 3375
3375 3384
3384 3430
3430 3481
3481 3492
3492 3585
3

6040 6193
6193 6281
6281 6605
6605 6848
selena gomez [201, 269, 354, 424, 497, 1065, 1472, 1548, 1596, 1689, 1848, 2245, 2248, 2267, 2304, 2411, 2656, 2848, 2925, 3166, 3188, 3256, 3533, 3536, 3589, 3668, 4185, 4541, 4553, 4557, 4577, 4748, 4749, 4793, 4845, 4897, 5101, 5157, 5164, 5180, 5196, 5418, 5453, 5516, 5670, 5722, 6021, 6263, 6293, 6298, 6393] 51
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 52
201 269
269 354
354 424
424 497
497 1065
1065 1472
1472 1548
1548 1596
1596 1689
1689 1848
1848 2245
2245 2248
2248 2267
2267 2304
2304 2411
2411 2656
2656 2848
2848 2925
2925 3166
3166 3188
3188 3256
3256 3533
3533 3536
3536 3589
3589 3668
3668 4185
4185 4541
4541 4553
4553 4557
4557 4577
4577 4748
4748 4749
4749 4793
4793 4845
4845 4897
4897 5101
5101 5157
5157 5164
5164 5180
5180 5196
5196 5418
5418 5453
5453 5516
5516 5670
5670 5722
5722 6021
6021 6263
6263 6293
6293 6298
6298 6393
khloe kardashian [213, 296, 468, 1216, 1749, 1932, 1987, 2000, 2201, 2640, 3406, 3504, 3660, 3844, 4560, 4620, 4628, 

In [44]:
new_all=np.array(new_all)

In [45]:
alldata=new_all

In [46]:
alldata.shape

(5748,)

In [47]:
target_list=[]
stem_list=[]
for data in alldata:
    target_list.append(data['text_aspect'])
    s=stemmer.stemWord(data['text_aspect'])
#     target_list.append(s)
#     stem_list
    stem_list.append(s)


In [48]:
target_dict=dict()
for i in range(len(alldata)):
    target=alldata[i]['text_aspect']
    if target not in target_dict:
        l=list()
        l.append(i)
        target_dict[target]=l
    else:
        l=target_dict[target]
        l.append(i)
        target_dict[target]=l

In [ ]:
target_dict

In [80]:
train_dict = {key: value for i, (key, value) in enumerate(target_dict.items()) if i % 40 != 0}
val_dict = {key: value for i, (key, value) in enumerate(target_dict.items()) if i %40 == 0}

In [81]:
train_dict

{'hilary swank': [1019,
  1020,
  1021,
  1022,
  1023,
  1024,
  1025,
  1026,
  1027,
  1028,
  1029,
  1030,
  1031,
  1032,
  1033,
  1034,
  1035,
  1036,
  1037,
  1038,
  1039,
  1040,
  1041,
  1042,
  1043,
  1044,
  1045,
  1046,
  1047,
  1048,
  1049,
  1050,
  1051,
  1052,
  1053,
  1054,
  1055,
  1056,
  1057,
  1058,
  1059,
  1060,
  1061,
  1062,
  1063,
  1064,
  1065,
  1066,
  1067,
  1068,
  1069,
  1070,
  1071,
  1072,
  1073,
  1074,
  1075,
  1076,
  1077,
  1078,
  1079,
  1080,
  1081,
  1082,
  1083,
  1084,
  1085,
  1086,
  1087,
  1088,
  1089,
  1090,
  1091,
  1092,
  1093],
 'wii': [1094,
  1095,
  1096,
  1097,
  1098,
  1099,
  1100,
  1101,
  1102,
  1103,
  1104,
  1105,
  1106,
  1107,
  1108,
  1109,
  1110,
  1111,
  1112,
  1113,
  1114,
  1115,
  1116,
  1117,
  1118,
  1119,
  1120,
  1121,
  1122,
  1123,
  1124,
  1125,
  1126,
  1127,
  1128,
  1129,
  1130,
  1131,
  1132,
  1133,
  1134,
  1135,
  1136,
  1137,
  1138,
  1139,
  1140,


In [82]:
val_dict

{'britney spears': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,


In [83]:
train_index=[]
print(len(train_dict))
for key,value in train_dict.items():
    train_index.extend(value)
len(train_index)

81


4679

In [84]:
val_index=[]
print(len(val_dict))
for key,value in val_dict.items():
    val_index.extend(value)
len(val_index)

3


1069

In [85]:
# np.save('train-restaurant.npy',alldata[train_index])
np.save('./datasets/resplit/train-twitter-pair.npy',alldata[train_index])

In [86]:
np.save('./datasets/resplit/valid-twitter-pair.npy',alldata[val_index])

In [ ]:
len(train_index)